# 準備
## 画像・動画データの用意
プロジェクトのrootにあたるディレクトリに、以下のディレクトリとファイルを用意する。
- `rep/`：代表画像を入れる。
- `back/`：背景の動画を入れる。
- `field/`：栗の映った動画を入れる。
- `masks/`：次節「アノテーションの作成」を参照。
- `labels.txt`: クラス名の一覧（とおまじない）を書く。

## アノテーションの作成
labelmeでセマンティックセグメンテーション用のラベルをつける。
labelmeが吐き出した`.json`ファイルを、VOC形式の`.npy`ファイルに変換する。この作業はcolabだと環境設定がよくわからないのでローカルでやってしまう。それを`masks/`に入れる。

# Colabの初期設定
ドライブをマウントしたら、以下を実行する。

In [1]:
%cd /content/drive/MyDrive/yolov5/
!source setup.sh

/content/drive/MyDrive/yolov5
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  emacs25 emacs25-bin-common emacs25-common emacs25-el emacsen-common
  fonts-droid-fallback fonts-noto-mono ghostscript gsfonts
  imagemagick-6-common install-info libcupsfilters1 libcupsimage2 libgpm2
  libgs9 libgs9-common libijs-0.35 libjbig2dec0 liblockfile-bin liblockfile1
  liblqr-1-0 libm17n-0 libmagickcore-6.q16-3 libmagickwand-6.q16-3 libotf0
  m17n-db poppler-data
Suggested packages:
  emacs25-common-non-dfsg ncurses-term fonts-noto ghostscript-x gpm m17n-docs
  libmagickcore-6.q16-3-extra gawk poppler-utils fonts-japanese-mincho
  | fonts-ipafont-mincho fonts-japanese-gothic | fonts-ipafont-gothic
  fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  emacs emacs25 emacs25-bin-common emacs25-common emacs25-el emacsen-common
  fonts-droid-fallback fonts-no

In [80]:
# 古い生成データを消去したければ以下を実行。
# !rm -r datasets/20211121/composite
# !rm -r datasets/20211121/domain_adaptation_1/

# 合成データの生成
生成するデータ数を`n_sample`として、クラスごとの発生確率を`prob`として設定する(`labels.txt`にリストアップされている順番で設定する)。また、第1段階のdomain adaptation用のデータを生成するかどうかを`domain_adaptation`として設定する。

In [10]:
# レポジトリに移動。
%cd chestnut-detection/

In [26]:
# ここを自由に設定。
n_sample = 20000
prob = [0.25, 0.25, 0.25, 0.25] # 空でもよい；代表画像内での出現頻度そのものになる
domain_adaptation = False
resume = True

以下を実行すると、前景画像を背景画像にランダムに貼り付けてできる`n_sample`枚の画像とそれに対応したラベルからなる人工データセット、さらに、第1段階のDomain adaptationに用いる、代表画像を回転させてできるデータセットが生成される。
GPUが利用できるときにはGPUを利用するようになっている(`--cuda`フラグ)。

In [27]:
prob_flag = '-p ' + ' '.join([str(p) for p in prob]) if prob else ''
domain_adaptation_flag = '--domain-adaptation domain_adaptation_1' if domain_adaptation else ''
resume_flag = '--resume' if resume else ''
!python make_dataset.py -n {n_sample} {prob_flag} --root ../datasets/20211121/ -o composite --verbose --bbox --cuda {domain_adaptation_flag} {resume_flag}

using cpu device
making output directories...done
getting class information...done
searching for where you left off...done
constructing asset objects...done
synthesizing dataset...
 79% 15879/20000 [02:36<3:22:42,  2.95s/it]
interrupted by keyboard


In [22]:
ls -l ../datasets/20211121/composite/images/all/ | wc -l

13389


fatal: could not read Username for 'https://github.com': No such device or address


In [42]:
# デコレータの練習
def print_return_value(func):
    def wrapper(*args, **kwargs):
        ret = func(*args, **kwargs)
        print(f'return value = {ret}')
        return ret
    return wrapper

@print_return_value
def add(a, b):
    return a + b

c = add(1, 2)
print(c)

return value = 3
3


In [39]:
# コンテクストマネジャの練習
class manager:
    def __init__(self):
        self.in_context = False

    def __enter__(self):
        print('enter context')
        self.in_context = True

    def __exit__(self, *args):
        if not all([arg is None for arg in args]):
            print(args)
        print('exit context')
        self.in_context = False
        return args[0] is ValueError

    def __repr__(self):
        return f'<context manager, status={"IN_CONTEXT" if self.in_context else "NOT_IN_CONTEXT"}>'

m = manager()
print(m)
with m:
    print(m)
    raise ValueError('message')
print(m)

<context manager, status=NOT_IN_CONTEXT>
enter context
<context manager, status=IN_CONTEXT>
(<class 'ValueError'>, ValueError('message'), <traceback object at 0x7fe82c4a08c0>)
exit context
<context manager, status=NOT_IN_CONTEXT>


In [57]:
from contextlib import contextmanager

@contextmanager
def managed_resource():
    # ここが__enter__の中身
    in_context = False
    print(f'inistantiated. in_context={in_context}')
    try:
        in_context = True
        print(f'enter context. in_context={in_context}')
        yield # __enter__の返り値. 今はNone
    except ValueError:
        pass
    finally:
        in_context = False
        print(f'exit context. in_context={in_context}')

m = managed_resource()
print('before with statement')
with m:
    print('suite')
    raise ValueError


before with statement
inistantiated. in_context=False
enter context. in_context=True
suite
exit context. in_context=False


In [202]:
class Variable:
    def __init__(self, val):
        self.set_val(val)
    def set_val(self, val):
        self.val = val
    def __repr__(self):
        return f'Variable({self.val})'
    @contextmanager
    def val_as(self, tmp_val):
        val = self.val
        try:
            self.set_val(tmp_val)
            yield
        finally:
            self.set_val(val)

var = Variable(0)
print(var)
with var.val_as(100):
    print(var)
print(var)

Variable(0)
Variable(100)
Variable(0)


[autoreload of _make_dataset failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
TypeError: 'module' object is not callable
]


In [204]:
var = Variable(1)
var.val_as(100)
print(var.val)

1


In [78]:
pwd

'/content/drive/My Drive/yolov5/chestnut-detection'

In [87]:
import simulate_dataset as sim
objects, cls_mask , obj_mask = sim.foreground_obj.from_voc(
    '../datasets/20211121/rep/WIN_20211112_14_48_16_Pro_0007.png', 
    '../datasets/20211121/masks/WIN_20211112_14_48_16_Pro_0007.npy', 
    classes
)

In [118]:
import dataclasses

@dataclasses.dataclass
class TensorContainer:
    tensor: torch.Tensor
    tensor_cropped: torch.Tensor = dataclasses.field(init=False)

    def __post_init__(self):
        self.tensor_cropped = self.tensor[1:, 1:]

x = torch.arange(12).view(3, 4)
tc1 = TensorContainer(x)
tc2 = TensorContainer(x)

In [120]:
tc1.tensor[:] = torch.ones((3, 4))
print(tc1)
print(tc2)

TensorContainer(tensor=tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]]), tensor_cropped=tensor([[1, 1, 1],
        [1, 1, 1]]))
TensorContainer(tensor=tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]]), tensor_cropped=tensor([[1, 1, 1],
        [1, 1, 1]]))


In [117]:
class A:
    def __init__(self):
        self.B = B()

class B:
    def __init__(self):
        self.C = C()

class C:
    def __init__(self):
        pass

A().B.C

In [222]:
from importlib import reload
import _make_dataset as mkdata
reload(mkdata)

p_video_dir = pathlib.Path('../datasets/20211121/chestnut')
videos = []
for p_video in p_video_dir.glob('*.mp4'):
    video = mkdata.FieldVideo(
        p_video=p_video,
        p_rep_images='../datasets/20211121/rep/',
        p_masks='../datasets/20211121/masks/',
        rep_image_extension='.png',
        classes=classes
    )
    videos.append(video)

for video in videos:
    print(bool(video.rep_images))

False
False
True
True
False
True


In [211]:
import dataclasses

@dataclasses.dataclass
class Super:
    a: int
    opt: int = dataclasses.field(init=False)

    def __post_init__(self):
        self.opt = 100

@dataclasses.dataclass
class Sub(Super):
    b: str = 'hello'
    # c: float = dataclasses.field(init=False)

    # def __post_init__(self):
    #     self.c = 1000

s = Sub(1, 'world')
print(s.a, s.b, s.opt)

1 world 100


In [232]:
import cv2

def get_n_frame(cap):
    n_frame = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    n_frame = int(n_frame)
    return n_frame

cap = cv2.VideoCapture('../datasets/20211121/back/WIN_20211112_12_31_27_Pro.mp4')
print(get_n_frame(cap))
cap.release()
print(get_n_frame(cap))
cap.open()
print(get_n_frame(cap))
cap.release()

451
0


TypeError: ignored

In [1]:
f = open('./utils.py')
print(f)
f.close()
f.read()

FileNotFoundError: ignored

In [23]:
from tqdm.auto import tqdm
import time

for i in tqdm(range(100)):
    time.sleep(0.1)

  0%|          | 0/100 [00:00<?, ?it/s]